In [1]:
import tensorflow as tf
import numpy as np

In [2]:
@tf.function
def add(a, b):
  return a + b
print(add(4,5))
aa = tf.constant(3)
bb = tf.constant(4)
cc = tf.constant(np.array([3,4]))
dd = tf.constant(np.array([4,5]))
print(add(aa,bb))
print(add(cc,dd))

tf.Tensor(9, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor([7 9], shape=(2,), dtype=int64)


In [83]:
n_dim = 3
BINS_MAX = 5
n_events = int(2e1)
DTYPE=tf.float64
DTYPEINT=tf.int64
x = tf.Variable(tf.zeros(n_dim, dtype=DTYPE))
div_index = tf.Variable(tf.zeros((n_dim), dtype=DTYPEINT))
reg_i = 0.0
reg_f = 1.0

def int_me(i):
    return tf.constant(i, dtype = DTYPEINT)
def float_me(i):
    return tf.constant(i, dtype = DTYPE)

ione = int_me(1)
izero = int_me(0)
fone = float_me(1)
fzero = float_me(0)

shape_rn = tf.TensorSpec(shape=(None,1), dtype=DTYPE)
shape_sub = tf.TensorSpec(shape=(None, BINS_MAX), dtype = DTYPE)

The first step is to be able to do the generation of the random numbers in a batched way

In [135]:
ione = int_me(1)
izero = int_me(0)
fone = float_me(1)
fzero = float_me(0)
n_events = 1000

shape_rn = tf.TensorSpec(shape=(None, n_dim), dtype=DTYPE)
shape_sub = tf.TensorSpec(shape=(n_dim, BINS_MAX), dtype = DTYPE)

@tf.function(input_signature=[shape_rn, shape_sub])
def gen_one_dim(rn, subdivision):
    """ Digest one random number
    """
    # Get the corresponding random number
    xn = BINS_MAX*(1.0 - rn)
    int_xn = tf.maximum(tf.cast(0, DTYPEINT),
                       tf.minimum(tf.cast(xn, DTYPEINT), BINS_MAX))
    # In practice int_xn = int(xn)-1 unless xn < 1
    aux_rand = xn - tf.cast(int_xn, dtype=DTYPE)
    # Now get the indices that will be used with this subdivision
    # If the index is 0, we cannot get the index-1 so...
    ind_f = tf.transpose(int_xn)
    ind_i = tf.maximum(ind_f - 1, 0)
    gather_f = tf.gather(subdivision, ind_f, batch_dims=1)
    gather_i_tmp = tf.gather(subdivision, ind_i, batch_dims=1)
    # Now the ones that had a "fake 0" need to be set to 0
    ind_is_0 = tf.equal(ind_f, 0)
    gather_i = tf.where(ind_is_0, fzero, gather_i_tmp)
    # Now compute the random number for this dimension
    x_ini = tf.transpose(gather_i)
    xdelta = tf.transpose(gather_f) - x_ini
    rand_x = x_ini + xdelta*aux_rand
    x = reg_i + rand_x*(reg_f - reg_i)
    weights = tf.reduce_prod(xdelta*BINS_MAX, axis=1)
    return x, int_xn, weights
oner = tf.random.uniform((n_events, n_dim), minval = 0.0, maxval=1.0, dtype=DTYPE)
#gen_one_dim(oner, divisions)
x, ind, w = gen_one_dim(oner, divisions)
print(divisions.shape)
print(f"IN shape: {oner.shape}")
print(f"OUT shape x: {x.shape}")
print(f"OUT shape index: {ind.shape}")
print(f"OUT shape weight: {w.shape}")

(3, 5)
IN shape: (1000, 3)
OUT shape x: (1000, 3)
OUT shape index: (1000, 3)
OUT shape weight: (1000,)


The same must be also possible for the integrand

In [136]:
@tf.function
def quick_integrand(xarr, n_dim = None):
    """Le page test function"""
    if n_dim is None:
        n_dim = xarr.shape[-1]
    a = tf.constant(0.1, dtype=DTYPE)
    n100 = tf.cast(100*n_dim, dtype=DTYPE)
    pref = tf.pow(1.0/a/np.sqrt(np.pi), n_dim)
    coef = fzero
    for i in range(n100+1):
        coef += i
    for i in range(n_dim):
        coef += tf.pow( (x[:,i]-1.0/2.0)/a, 2)
    coef -= (n100+1)*n100/2.0
    return pref*tf.exp(-coef)
tmp = quick_integrand(x, n_dim=n_dim)/n_events
tmp2 = tf.square(tmp)

And, finally, the same must also be true for the creation of the array of partial results per BIN

In [243]:
my_dim = 2
all_ind = ind[:,0:my_dim]

In [255]:
%%time 
np_i = all_ind.numpy()
np_tmp2 = tmp2.numpy()
arr_res2 = np.zeros((my_dim, BINS_MAX))
for i in range(n_events):
    for j in range(my_dim):
        arr_res2[j, np_i[i, j]] += tmp2[i]

CPU times: user 1.9 s, sys: 2.36 ms, total: 1.91 s
Wall time: 1.91 s


In [210]:
%%time
if my_dim == 1:
    arr_res2 = []
    for i in range(BINS_MAX):
        mask = tf.equal(all_ind, i)[:,0]
        res_tmp = tf.reduce_sum(tf.boolean_mask(tmp2, mask))
        arr_res2.append(res_tmp.numpy())

CPU times: user 127 ms, sys: 3.22 ms, total: 130 ms
Wall time: 127 ms


In [258]:
%%time
def consume_events(res2, ind):
    arr_res2 = []
    all_bins = tf.range(BINS_MAX, dtype = DTYPEINT)
    for j in range(my_dim):
        eq = tf.transpose(tf.equal(ind[:,j:j+1], all_bins))
        res_tmp = tf.reduce_sum(tf.where(eq, res2, fzero))
        arr_res2.append(res_tmp)
    return arr_res2
xx = consume_events(tmp2, all_ind)

CPU times: user 28.6 ms, sys: 3.24 ms, total: 31.8 ms
Wall time: 26.6 ms


In [252]:
all_bins = tf.range(BINS_MAX, dtype = DTYPEINT)
mask = tf.transpose(tf.equal(all_ind[:,0:1], all_bins))
res_tmp = tf.where(mask, tmp2, fzero)
res_tmp.shape

TensorShape([5, 1000])

In [246]:
tf.equal(all_ind[:,0:1], all_bins)

<tf.Tensor: id=168662, shape=(1000, 5), dtype=bool, numpy=
array([[False, False, False, False,  True],
       [False, False, False, False,  True],
       [False,  True, False, False, False],
       ...,
       [False, False, False, False,  True],
       [False, False, False,  True, False],
       [False, False, False, False,  True]])>